# Imports and Paths

In [1]:
import pandas as pd
import json

In [14]:
import sys
sys.path.append('../../fastai/')
from fastai.learner import *
from fastai.column_data import *

NameError: name 'Dataset' is not defined

In [9]:
import torch

In [ ]:
torch.

# Import Data and reshape

In [2]:
with open('../data/all_game_dict.json') as fp:
    all_game_dict = json.load(fp)
with open('../data/all_game_dict_v2.json') as fp:
    all_game_dict_v2 = json.load(fp)
with open('../data/all_game_dict_v3.json') as fp:
    all_game_dict_v3 = json.load(fp)
with open('../data/all_game_dict_v4.json') as fp:
    all_game_dict_v4 = json.load(fp)
all_game_dict.update(all_game_dict_v2)
all_game_dict.update(all_game_dict_v3)
all_game_dict.update(all_game_dict_v4)

In [13]:
all_users_dict = {}
for kys1 in all_game_dict.keys():
    for kys2 in all_game_dict[kys1].keys():
        if kys2 not in all_users_dict:
            all_users_dict[kys2] = {kys1:all_game_dict[kys1][kys2]}
        else:
            all_users_dict[kys2].update({kys1:all_game_dict[kys1][kys2]})

In [38]:
min_rat = 50
hi_rating_user_dict = {}
for kys in all_users_dict.keys():
    if len(all_users_dict[kys]) >= min_rat:
        hi_rating_user_dict[kys] = all_users_dict[kys]
len(all_users_dict), len(hi_rating_user_dict)

(243525, 47388)

In [ ]:
flat_hi_user_rat_dict = {}
for kys in hi_rating_user_dict.keys():
    

In [45]:
df = pd.DataFrame.from_dict(hi_rating_user_dict, orient='index')
df.shape

(47388, 2000)

In [53]:
df_melt = df.reset_index()
df_melt = df_melt.rename(columns={'index':'user'})
df_melt = pd.melt(df_melt, id_vars='index', var_name='game_id', value_name='rating')
df_melt = df_melt.dropna().reset_index(drop=True)

In [64]:
df_melt.head()

,user,game_id,rating
0,-=Yod@=-,174430,10
1,-mIDE-,174430,9
2,0492372665,174430,9
3,0b1_Ita,174430,8
4,0xDEADBEEF,174430,10


Invert json to give recommendations for each user, instead of for each game

In [65]:
df_melt.to_csv('../data/user_ratings_gt50_melt.csv')

In [46]:
df.to_csv('../data/user_ratings_gt50.csv')

In [49]:
df.loc['protect3f'].notnull().sum()

72